In [1]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import h5py
import gc

In [2]:
code_path = r'Data/CODE-15% a large scale annotated dataset of 12-lead ECGs/'
print(code_path + 'Finalized Data/' + 'AF_traced')

Data/CODE-15% a large scale annotated dataset of 12-lead ECGs/Finalized Data/AF_traced


In [3]:
# display head of csv file
df = pd.read_csv(code_path + 'exams.csv')
df.head()

,exam_id,age,is_male,nn_predicted_age,1dAVb,RBBB,LBBB,SB,ST,AF,patient_id,death,timey,normal_ecg,trace_file
0,1169160,38,True,40.160484,False,False,False,False,False,False,523632,False,2.098628,True,exams_part13.hdf5
1,2873686,73,True,67.059440,False,False,False,False,False,False,1724173,False,6.657529,False,exams_part13.hdf5
2,168405,67,True,79.621740,False,False,False,False,False,True,51421,False,4.282188,False,exams_part13.hdf5
3,271011,41,True,69.750260,False,False,False,False,False,False,1737282,False,4.038353,True,exams_part13.hdf5
4,384368,73,True,78.873460,False,False,False,False,False,False,331652,False,3.786298,False,exams_part13.hdf5


In [4]:
#print exam id where AF is TRUE
af = df[df['AF'] == True]['exam_id']
davb = df[df['1dAVb'] == True]['exam_id']
rbbb = df[df['RBBB'] == True]['exam_id']
lbbb = df[df['LBBB'] == True]['exam_id']
sb = df[df['SB'] == True]['exam_id']
st = df[df['ST'] == True]['exam_id']
print(len(af), len(davb), len(rbbb), len(lbbb),len(sb),len(st), len(df),len(df)-(len(af) + len(davb) + len(rbbb) + len(sb) + len(st) + len(lbbb)))
# print with proper text formatting
print('AF: ', len(af), '\n1dAVb: ', len(davb), '\nRBBB: ', len(rbbb), '\nLBBB: ', len(lbbb), '\nSB: ', len(sb), '\nST: ', len(st), '\nTotal: ', len(df), '\nOther: ', len(df)-(len(af) + len(davb) + len(rbbb) + len(sb) + len(st) + len(lbbb)))
#print exam id where all is false
none = df[(df['AF'] == False) & (df['1dAVb'] == False) & (df['RBBB'] == False) & (df['LBBB'] == False) & (df['SB'] == False) & (df['ST'] == False)]
print('None: ', len(none))

7033 5716 9672 6026 5605 7584 345779 304143
AF:  7033 
1dAVb:  5716 
RBBB:  9672 
LBBB:  6026 
SB:  5605 
ST:  7584 
Total:  345779 
Other:  304143
None:  308004


In [5]:
af_trace = df[df['AF'] == True]['trace_file']
#combine af and af_trace
af_with_trace = pd.concat([af, af_trace], axis=1)

In [6]:
print(af_with_trace.head())
for i in range(10):

    print(af_with_trace.iloc[i,1])

    exam_id         trace_file
2    168405  exams_part13.hdf5
9    132538  exams_part13.hdf5
57  2821208  exams_part13.hdf5
74   669209  exams_part14.hdf5
77   360491  exams_part14.hdf5
exams_part13.hdf5
exams_part13.hdf5
exams_part13.hdf5
exams_part14.hdf5
exams_part14.hdf5
exams_part13.hdf5
exams_part10.hdf5
exams_part14.hdf5
exams_part13.hdf5
exams_part0.hdf5


In [7]:
def normalize(x):
    x = (x - x.min()) / (x.max() - x.min())
    return x

In [ ]:
# load the hdf5 file from the first column of the af_with_trace dataframe
trace_dataset = []
exam_id_dataset = []
for i in range(len(af_with_trace)):
    temp_str = af_with_trace.iloc[i,1].split('.')[0]+'/'
    # print(temp_str)
    file = h5py.File(code_path + temp_str + af_with_trace.iloc[i,1], 'r')
    # print(file.keys(),file.name)
    #get tracing from opened hdf5 where exam_id is equal to the first column of af_with_trace 
    temp_index = np.where(file['exam_id'][:] ==  af_with_trace.iloc[i,0])[0][0]
    print(temp_index,end = ' ')
    exam_id = file['exam_id'][temp_index]
    exam_id_dataset.append(exam_id)
    trace = file['tracings'][temp_index]
    trace_dataset.append(normalize(trace))
    print(trace.shape,end = '\n')
    # plt.figure(figsize=(20,7))
    # for j in range(0,12):
    #     plt.subplot(12,1,j+1)
    #     plt.plot(np.array(trace_dataset[i][:,j]))
    file.close()
    # write data to a new hdf5 file which is previously created
    

In [9]:
trace_dataset = np.array(trace_dataset)
exam_id_dataset = np.array(exam_id_dataset)
print(trace_dataset.shape,exam_id_dataset.shape)
# for i in range(len(af_with_trace)):
os.mkdir(code_path + 'Finalized Data/')
with h5py.File(code_path + 'Finalized Data/' + 'AF_traced', 'a') as file:
    file.create_dataset('exam_id', data=exam_id_dataset)
    file.create_dataset('tracings', data=trace_dataset)
    file.close()
del trace_dataset, exam_id_dataset
gc.collect()
#create readme
readme = open(code_path + 'Finalized Data/' + 'af_dataset_readme.txt','w')
readme.write('This is a dataset of 12-lead ECGs with atrial fibrillation. \n') 
readme.write('The data is from the CODE-15% dataset. \n')
readme.write('The data is normalized to a range of 0 to 1. \n')
readme.write('The data is stored in a hdf5 file. \n')
readme.write('The data is stored in two datasets: exam_id and tracings. \n')
readme.write('The exam_id dataset is a 1D array of exam_id. \n')
readme.write('The tracings dataset is a 3D array of tracings. \n')
readme.close()

(7033, 4096, 12) (7033,)
